In [1]:
# FoML Assignment-3
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Enter Your Name Here
myname = "Rohan B M" 
Roll_No = "BM21MTECH14003"

# Question No.5a - Pre-Processing the popular Lending Club dataset
#load the dataset
df = pd.read_csv('loan_train.csv')
df.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,573354,737474,10000,10000,9950.00000,36 months,7.51%,311.11,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,476321,603324,15000,15000,14800.00000,36 months,8.94%,476.58,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,451484,556265,2000,2000,2000.00000,36 months,13.57%,67.94,C,C3,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1018129,1246557,35000,35000,33951.84413,60 months,20.89%,944.71,F,F1,...,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN
4,800018,1005270,14000,14000,14000.00000,60 months,17.49%,351.64,D,D5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


In [2]:
df.shape

(24999, 111)

In [3]:
df['loan_status'].value_counts()

Fully Paid     20827
Charged Off     3474
Current          698
Name: loan_status, dtype: int64

In [4]:
df = df.loc[df['loan_status'].isin(['Fully Paid','Charged Off'])]
df['loan_status'].value_counts(normalize=False, dropna=False)

Fully Paid     20827
Charged Off     3474
Name: loan_status, dtype: int64

In [5]:
df.shape

(24301, 111)

In [6]:
df['loan_status'] = df['loan_status'].apply(lambda x : +1 if x=='Fully Paid' else -1)


In [7]:
df['loan_status'].unique()

array([-1,  1], dtype=int64)

In [8]:
df = df[df.columns[((df.isnull().sum())/len(df)) < 0.4]]
df.shape

(24301, 54)

In [9]:
df.drop(['emp_title','pymnt_plan','url','desc','title', 'zip_code', 'addr_state', 'initial_list_status','application_type',],axis=1,inplace=True)

In [10]:
df.drop(['id','member_id','issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d','acc_now_delinq','chargeoff_within_12_mths','sub_grade','revol_util',
         'collections_12_mths_ex_med','delinq_amnt','policy_code','tax_liens','out_prncp','out_prncp_inv'],axis=1,inplace=True)

In [11]:
pd.set_option('display.max_columns', None)

df.head(2)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,10000,10000,9950.0,36 months,7.51%,311.11,A,10+ years,RENT,30000.0,Source Verified,-1,home_improvement,5.00,0,3,19,0,1450,48,2247.21,2235.98,1509.91,350.87,0.0,386.43,3.96,311.11,0.0
1,15000,15000,14800.0,36 months,8.94%,476.58,A,1 year,RENT,147000.0,Not Verified,1,other,3.47,0,0,6,0,4910,17,15112.76,14911.26,15000.00,112.76,0.0,0.00,0.00,15114.03,0.0


In [12]:
df.shape

(24301, 29)

In [13]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.home_ownership)
df['home_ownership']=le.transform(df.home_ownership)

le.fit(df.verification_status)
df['verification_status']=le.transform(df.verification_status)

le.fit(df.purpose)
df['purpose']=le.transform(df.purpose)

In [14]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.grade)
df['grade']=le.transform(df.grade)

le.fit(df.term)
df['term']=le.transform(df.term)

In [15]:
# Converting int_rate and emp-length to numerical columns
df.int_rate = df.int_rate.apply(lambda x:x[:2])
df.int_rate = df.int_rate.astype('float')

In [16]:
df.pub_rec_bankruptcies.value_counts()

0.0    22886
1.0      994
2.0        4
Name: pub_rec_bankruptcies, dtype: int64

In [17]:
df['pub_rec_bankruptcies'].fillna(df['pub_rec_bankruptcies'].median(),inplace=True)
df['pub_rec_bankruptcies'].unique()

array([0., 1., 2.])

In [18]:
# seperating null values by assigning a random string
df['emp_length'].fillna('5000',inplace=True) 
# Filling less than one year as 0 years of experience
df.emp_length.replace({'10+ years':'10 years','< 1 year':'0 years'},inplace=True) 
# Converting it's dtype to float
df.emp_length = df.emp_length.apply(lambda x:x[:2])
df.emp_length = df.emp_length.astype('float')

In [19]:
# Building models to fill null values in emp_length columns
df2 = df.drop('loan_status',axis=1)
df2_train = df2[df2['emp_length'] !=50]
df2_test = df2[df2['emp_length'] ==50]
# Assigning train and test to variables
x_train = df2_train.drop('emp_length',axis=1)
y_train = df2_train.emp_length
x_test = df2_test.drop('emp_length',axis=1)

In [20]:
df2_train.drop('emp_length',axis=1)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,10000,10000,9950.00000,0,7.0,311.11,0,3,30000.0,1,4,5.00,0,3,19,0,1450,48,2247.210000,2235.98,1509.91,350.87,0.0,386.43,3.96,311.11,0.0
1,15000,15000,14800.00000,0,8.0,476.58,0,3,147000.0,0,9,3.47,0,0,6,0,4910,17,15112.760000,14911.26,15000.00,112.76,0.0,0.00,0.00,15114.03,0.0
2,2000,2000,2000.00000,0,13.0,67.94,2,2,36000.0,0,6,7.83,0,0,8,0,1790,10,2354.966827,2354.97,2000.00,354.97,0.0,0.00,0.00,101.78,0.0
4,14000,14000,14000.00000,1,17.0,351.64,3,0,50000.0,0,2,21.24,1,1,9,1,553,27,20804.230020,20804.23,14000.00,6804.23,0.0,0.00,0.00,3943.27,0.0
5,12000,12000,11900.00000,0,8.0,381.26,0,0,294000.0,0,9,0.50,0,0,11,0,5306,21,12344.811770,12241.94,12000.00,344.81,0.0,0.00,0.00,11204.30,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,5500,5500,5500.00000,0,14.0,190.55,3,0,52000.0,0,2,18.74,0,2,11,0,10655,37,6825.066783,6825.07,5500.00,1325.07,0.0,0.00,0.00,1122.20,0.0
24995,11450,11450,11450.00000,0,8.0,361.40,0,3,40000.0,1,2,21.21,0,0,10,0,10192,15,13010.206030,13010.21,11450.00,1560.21,0.0,0.00,0.00,382.90,0.0
24996,24000,24000,21100.31424,1,20.0,642.96,6,0,74454.0,2,2,19.95,0,3,9,0,29108,24,33765.068310,26978.96,24000.00,9765.07,0.0,0.00,0.00,16500.39,0.0
24997,14000,14000,11411.12089,0,17.0,503.19,5,0,86000.0,2,2,21.93,0,0,13,1,18991,26,17917.012900,14191.06,14000.00,3917.01,0.0,0.00,0.00,3857.88,1.0


In [21]:
tar_col=df['loan_status']
df.drop(['loan_status'],axis=1,inplace=True)
df=df.join(tar_col)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24301 entries, 0 to 24998
Data columns (total 29 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_amnt                24301 non-null  int64  
 1   funded_amnt              24301 non-null  int64  
 2   funded_amnt_inv          24301 non-null  float64
 3   term                     24301 non-null  int32  
 4   int_rate                 24301 non-null  float64
 5   installment              24301 non-null  float64
 6   grade                    24301 non-null  int32  
 7   emp_length               24301 non-null  float64
 8   home_ownership           24301 non-null  int32  
 9   annual_inc               24301 non-null  float64
 10  verification_status      24301 non-null  int32  
 11  purpose                  24301 non-null  int32  
 12  dti                      24301 non-null  float64
 13  delinq_2yrs              24301 non-null  int64  
 14  inq_last_6mths        

In [22]:
pd.set_option('display.max_columns', None)

df.head(20)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,total_acc,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies,loan_status
0,10000,10000,9950.000000,0,7.0,311.11,0,10.0,3,30000.0,1,4,5.00,0,3,19,0,1450,48,2247.210000,2235.98,1509.91,350.87,0.000000,386.43,3.9600,311.11,0.0,-1
1,15000,15000,14800.000000,0,8.0,476.58,0,1.0,3,147000.0,0,9,3.47,0,0,6,0,4910,17,15112.760000,14911.26,15000.00,112.76,0.000000,0.00,0.0000,15114.03,0.0,1
2,2000,2000,2000.000000,0,13.0,67.94,2,4.0,2,36000.0,0,6,7.83,0,0,8,0,1790,10,2354.966827,2354.97,2000.00,354.97,0.000000,0.00,0.0000,101.78,0.0,1
4,14000,14000,14000.000000,1,17.0,351.64,3,5.0,0,50000.0,0,2,21.24,1,1,9,1,553,27,20804.230020,20804.23,14000.00,6804.23,0.000000,0.00,0.0000,3943.27,0.0,1
5,12000,12000,11900.000000,0,8.0,381.26,0,7.0,0,294000.0,0,9,0.50,0,0,11,0,5306,21,12344.811770,12241.94,12000.00,344.81,0.000000,0.00,0.0000,11204.30,0.0,1
6,10000,6525,6480.473624,1,7.0,131.93,0,4.0,0,65000.0,0,1,7.62,1,1,13,0,5629,23,7908.889997,7846.07,6525.00,1383.89,0.000000,0.00,0.0000,668.53,0.0,1
7,7800,7800,7800.000000,1,14.0,185.20,3,4.0,0,94000.0,2,9,12.96,0,1,11,0,2317,37,9864.289537,9864.29,7800.00,2064.29,0.000000,0.00,0.0000,58.59,0.0,1
8,18825,18825,18825.000000,0,7.0,589.04,0,5.0,3,42000.0,0,2,22.74,0,0,7,0,22121,22,20958.528940,20958.53,18825.00,2133.53,0.000000,0.00,0.0000,239.16,0.0,1
10,2600,2600,2600.000000,0,12.0,87.02,1,2.0,3,40792.0,0,6,13.54,0,1,12,0,9847,14,3132.393851,3132.39,2600.00,532.39,0.000000,0.00,0.0000,94.26,0.0,1
11,20000,20000,19725.000000,1,13.0,461.34,2,8.0,0,100000.0,2,2,12.98,0,1,15,0,20984,20,27679.645760,27299.05,20000.00,7679.65,0.000000,0.00,0.0000,513.18,0.0,1


In [23]:
tar_col

0       -1
1        1
2        1
4        1
5        1
        ..
24994    1
24995    1
24996    1
24997    1
24998    1
Name: loan_status, Length: 24301, dtype: int64

In [24]:
df.shape

(24301, 29)

In [25]:
df.to_csv('PP_traindata.csv',index=False)